In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## training yolov8 with dogs and cats dataset

https://www.kaggle.com/datasets/andrewmvd/dog-and-cat-detection

#### download dataset from ```https://github.com/sergiovirahonda/FaceMaskDataset/tree/main ```

In [ ]:
!git clone https://github.com/sergiovirahonda/FaceMaskDataset.git

In [ ]:
!pip install ultralytics

#### CLI

In [ ]:
!yolo detect train data=/content/FaceMaskDataset/dataset.yaml model=yolov8n.pt epochs=10 imgsz=320

#### python test

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/runs/detect/train/weights/best.pt')
model.predict("/content/FaceMaskDataset/images/train/0.jpg", save=True, imgsz=320, conf=0.5)

## export yolov8 to trt

In [ ]:
from pathlib import Path
# your model path
train_path = Path("/content/runs/detect/train")
model_path = train_path / "weights"
pt_file = str(model_path / "best.pt")
onnx_file = str(model_path / "best.onnx")
trt_file = str(model_path / "best.engine")

In [ ]:
# from google.colab import files
# files.download(train_path / "results.png")

In [ ]:
!yolo export model={pt_file} format=onnx

In [ ]:
from google.colab import files
files.download(onnx_file)

### onnx to trt engine

In [ ]:
!python3 -m pip install --upgrade tensorrt

In [ ]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
def build_engine(onnx_path, shape):

   """
   This is the function to create the TensorRT engine
   Args:
      onnx_path : Path to onnx_file.
      shape : Shape of the input of the ONNX file.
  """
   with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, builder.create_builder_config() as config, trt.OnnxParser(network, TRT_LOGGER) as parser:
       config.max_workspace_size = (256 << 20)
       with open(onnx_path, 'rb') as model:
           parser.parse(model.read())
       network.get_input(0).shape = shape
       engine = builder.build_engine(network, config)
       return engine

def save_engine(engine, file_name):
   buf = engine.serialize()
   with open(file_name, 'wb') as f:
       f.write(buf)
def load_engine(trt_runtime, plan_path):
  with open(plan_path, 'rb') as f:
    engine_data = f.read()
  engine = trt_runtime.deserialize_cuda_engine(engine_data)
  # engine = trt_runtime.deserialize_cuda_engine(Path(plan_path).read_bytes())
  return engine

In [ ]:
import argparse
from onnx import ModelProto
import tensorrt as trt

engine_name = trt_file
onnx_path = onnx_file

batch_size = 1

model = ModelProto()
with open(onnx_path, "rb") as f:
    model.ParseFromString(f.read())

d0 = model.graph.input[0].type.tensor_type.shape.dim[1].dim_value
d1 = model.graph.input[0].type.tensor_type.shape.dim[2].dim_value
d2 = model.graph.input[0].type.tensor_type.shape.dim[3].dim_value
shape = [batch_size , d0, d1 ,d2]
engine = build_engine(onnx_path, shape= shape)
save_engine(engine, engine_name)

In [ ]:
output_0 = model.graph.output[0].type.tensor_type.shape.dim[0].dim_value
output_1 = model.graph.output[0].type.tensor_type.shape.dim[1].dim_value
output_2 = model.graph.output[0].type.tensor_type.shape.dim[2].dim_value
model_output_shape = (output_0 , output_1 , output_2)
print(model_output_shape)

## tensorRT inference yolov8 engine

In [ ]:
!pip install pycuda
!wget https://images.livemint.com/img/2023/02/15/1600x900/face_masks_1676434071914_1676434072173_1676434072173.jpg -O ./people.jpeg

In [ ]:
import tensorrt as trt
import pycuda.driver as cuda
import numpy as np
import pycuda.autoinit

def allocate_buffers(engine, batch_size, data_type):

   """
   This is the function to allocate buffers for input and output in the device
   Args:
      engine : The path to the TensorRT engine.
      batch_size : The batch size for execution time.
      data_type: The type of the data for input and output, for example trt.float32.

   Output:
      h_input_1: Input in the host.
      d_input_1: Input in the device.
      h_output_1: Output in the host.
      d_output_1: Output in the device.
      stream: CUDA stream.

   """

   # Determine dimensions and create page-locked memory buffers (which won't be swapped to disk) to hold host inputs/outputs.
   h_input_1 = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(0)), dtype=trt.nptype(data_type))
   h_output = cuda.pagelocked_empty(batch_size * trt.volume(engine.get_binding_shape(1)), dtype=trt.nptype(data_type))
   # Allocate device memory for inputs and outputs.
   d_input_1 = cuda.mem_alloc(h_input_1.nbytes)

   d_output = cuda.mem_alloc(h_output.nbytes)
   # Create a stream in which to copy inputs/outputs and run inference.
   stream = cuda.Stream()
   return h_input_1, d_input_1, h_output, d_output, stream

def load_images_to_buffer(pics, pagelocked_buffer):
   preprocessed = np.asarray(pics).ravel()
   np.copyto(pagelocked_buffer, preprocessed)

def do_inference(engine, pics_1, h_input_1, d_input_1, h_output, d_output, stream, batch_size, height, width):
   """
   This is the function to run the inference
   Args:
      engine : Path to the TensorRT engine
      pics_1 : Input images to the model.
      h_input_1: Input in the host
      d_input_1: Input in the device
      h_output_1: Output in the host
      d_output_1: Output in the device
      stream: CUDA stream
      batch_size : Batch size for execution time
      height: Height of the output image
      width: Width of the output image

   Output:
      The list of output images

   """

   load_images_to_buffer(pics_1, h_input_1)

   with engine.create_execution_context() as context:
      # Transfer input data to the GPU.
      cuda.memcpy_htod_async(d_input_1, h_input_1, stream)

      # Run inference.

      context.profiler = trt.Profiler()
      context.execute(batch_size=1, bindings=[int(d_input_1), int(d_output)])

      # Transfer predictions back from the GPU.
      cuda.memcpy_dtoh_async(h_output, d_output, stream)
      # Synchronize the stream
      stream.synchronize()
      # Return the host output.
      out = h_output.reshape((model_output_shape))
      # out = h_output
      return out


In [ ]:
def draw_detect(img , x , y , width , height , conf , label):
  # label = f'{CLASSES[class_id]} ({confidence:.2f})'
  # color = colors[class_id]
  print(x , y , width , height , conf , label)
  cv2.rectangle(img, (x, y), (x + width, y + height), (0,0,255), 2)
  cv2.putText(img, f"{label} {conf}", (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

def show_detect(img , preds , threshold = 0.5):
  boxes = []
  scores = []
  class_ids = []

  for pred_idx in range(preds.shape[2]):
    pred = preds[0,:,pred_idx]
    box = [pred[0] - 0.5*pred[2] , pred[1] - 0.5*pred[3] , pred[2] , pred[3]]
    conf = pred[4:]
    label = np.argmax(conf)
    max_conf = np.max(conf)
    # print(np.max(conf))


    boxes.append(box)
    scores.append(max_conf)
    class_ids.append(label)

  result_boxes = cv2.dnn.NMSBoxes(boxes, scores, 0.25, 0.45, 0.5)

  for i in range(len(result_boxes)):
    index = result_boxes[i]
    box = boxes[index]
    detection = {
            'class_id': class_ids[index],
            # 'class_name': CLASSES[class_ids[index]],
            'confidence': scores[index],
            'box': box,
            # 'scale': scale}
    }
    # detections.append(detection)
    draw_detect(img, round(box[0]), round(box[1]),round(box[2]), round(box[3]),
        scores[index] , class_ids[index])
  cv2_imshow(img)





In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import tensorrt as trt
import numpy as np
from PIL import Image
import tensorrt as trt
import torch.nn as nn

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)

serialized_plan_fp32 = trt_file

HEIGHT = 320
WIDTH = 320

img = cv2.imread("/content/people.jpeg")
img = cv2.resize(img , (WIDTH , HEIGHT))
im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
im = np.array(im, dtype=np.float32, order='C')
im = im.transpose((2, 0, 1))
im = (2.0 / 255.0) * im - 1.0

engine = load_engine(trt_runtime, serialized_plan_fp32)
h_input, d_input, h_output, d_output, stream = allocate_buffers(engine, 1, trt.float32)



In [ ]:
out = do_inference(engine, im, h_input, d_input, h_output, d_output, stream, 1, HEIGHT, WIDTH)
# cv2_imshow(img)
show_detect(img , out)